In [1]:
#export
from k1lib.callbacks import Callback, Callbacks
import k1lib as k1lib
import matplotlib.pyplot as plt
from functools import partial

In [2]:
#export
class ParamData(k1lib.Object):
    def __init__(self):
        super().__init__()
        self.means = []; self.stds = []
        self.mins = []; self.maxs = []
    def update(self, torchParam):
        self.means.append(torchParam.mean().item())
        self.stds.append(torchParam.std().item())
        self.mins.append(torchParam.min().item())
        self.maxs.append(torchParam.max().item())
    def __len__(self): return len(self.means)
    def __repr__(self):
        return f"""Param's saved data. Use...
- d.means: to get list of means
- d.stds: to get list of means
- d.mins: to get list of mins
- d.maxs: to get list of maxs"""
class Param:
    def __init__(self, name, torchParam):
        self.name = name
        self.torchParam = torchParam
        self.data = ParamData()
    def update(self): self.data.update(self.torchParam.detach())
    def __repr__(self):
        return f"""Param `{self.name}`. Use...
- p.torchParam: to get actual underlying parameter
- p.data: to get data stored
- cb.plot(): to quickly look at everything"""

In [3]:
#export
@k1lib.patch(Callback.cls)
class HookParam(Callback):
    """Records means and stds of all parameters"""
    def __init__(self): super().__init__(); self.params = []
    def __getitem__(self, idx):
        if type(idx) == int: return self.params[idx]
        answer = HookParam(); answer.params = self.params[idx]; return answer
    def __len__(self): return len(self.params)
    def _selected(self, name):
        splits = name.split(".")
        try:
            mS = self.selector
            for split in splits[:-1]: mS = mS[split]
            return "all" in mS.selectedProps or "HookParam" in mS.selectedProps
        except KeyError: return False
    def startRun(self):
        if len(self) == 0: # set things up first time only
            self.params = [Param(k, v) for k, v in self.model.named_parameters() if self._selected(k)]
    def startBatch(self): [param.update() for param in self.params]
    def __repr__(self):
        s = f", {len(self[0].data)} means and stds each" if len(self) > 0 else ""
        names = "\n".join([f"  {i}. {p.name}" for i, p in enumerate(self)])
        return f"""{super()._reprHead}: {len(self)} params{s}:\n{names}\n
Use...
- cb.plot(): to quickly look at everything
- cb[i]: to view a single param
- cb[a:b]: to get a new HookParam with selected params
{super()._reprCan}"""

In [4]:
#export
def plotF(params, rangeSlice):
    if type(params) == Param: params = [params]
    fields = params[0].data.state.keys()
    fig, axes = plt.subplots(2, 2, figsize=(10, 6), dpi=100)
    axes = axes.flatten()
    for field, ax in zip(fields, axes):
        for param in params:
            r = k1lib.Range(len(fieldData := param.data[field]))[rangeSlice]
            ax.plot(r.range, fieldData[r.slice])
        ax.set_title(field.capitalize())
    plt.figlegend([p.name for p in params], loc='right')
@k1lib.patch(HookParam)
@k1lib.patch(Param)
def plot(self): return k1lib.viz.SliceablePlot(partial(plotF, self))

In [5]:
#export
@k1lib.patch(Callbacks, docs=HookParam)
def withHookParam(self): return self.append(HookParam())

In [6]:
!../../export.py _callbacks/hookParam

Current dir: /home/kelvin/repos/labs/k1lib, ../../export.py
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 0.1.3
Uninstalling k1lib-0.1.3:
  Successfully uninstalled k1lib-0.1.3
running install
running bdist_egg
running egg_info
creating k1lib.egg-info
writing k1lib.egg-info/PKG-INFO
writing dependency_links to k1lib.egg-info/dependency_links.txt
writing requirements to k1lib.egg-info/requires.txt
writing top-level names to k1lib.egg-info/top_level.txt
writing manifest file 'k1lib.egg-info/SOURCES.txt'
reading manifest file 'k1lib.egg-info/SOURCES.txt'
writing manifest file 'k1lib.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/k1lib
copying k1lib/_learner.py -> build/lib/k1lib
copying k1lib/callbacks.py -> build/lib/k1lib
copying k1lib/data.py -> build/lib/k1lib
copying k1lib/imports.py -> build/lib/k1lib
copying k1lib/_